In [ ]:
from imports import *

In [ ]:
k = lambda x: x.split('/')[3]
k2 = lambda x: k(x)[:-4]
q = lambda fn: int(k(fn)[-3:])
ap = lambda fn: load(fn)['results']['bbox']['AP']
files = glob.glob('detectron2/data/*/*/results.json')
d = dict(((key, list(val)) for key, val in groupby(sorted(files, key=k), key=k2))); # d

In [ ]:
pd.set_option("display.precision", 1)
def get_df(selection=None):
    return pd.DataFrame({
        model.replace('evaluator_dump_', '').replace('_x1_big_ilsvrc', '').replace('_x1_small_zoo', '_FT').replace('_', ' '): {
            q(fn): ap(fn) for fn in files
        }
        for model, files in d.items()
        if len(files) > 1 and '_x3' not in model and (selection is None or model in selection)
    })
df = get_df(); df

In [ ]:
%matplotlib notebook
df.plot()

In [ ]:
from ipywidgets import interact, SelectMultiple, Layout

In [ ]:
@interact(sel=SelectMultiple(options=d.keys(), rows=len(d), layout=Layout(width='500px')))
def plot(sel):
    if not sel:
        print('Empty')
        return
    get_df(sel).plot(legend=True, xlabel='JPEG quality (Q)', ylabel='AP score (%)')